In [ ]:
from fyers_apiv3 import fyersModel
from fyers_apiv3.FyersWebsocket import data_ws
import pandas as pd
import time
from datetime import date

# Replace these values with your actual API credentials
client_id = "PP860PBEU0-100"
secret_key = "JC501DPXH7"
redirect_uri = "https://google.com"
response_type = "code"  
state = "sample_state"
session = fyersModel.SessionModel(
    client_id=client_id,
    secret_key=secret_key,
    redirect_uri=redirect_uri,
    response_type=response_type
)
response = session.generate_authcode()
print(response)

In [ ]:
auth_code = "eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJpc3MiOiJhcGkubG9naW4uZnllcnMuaW4iLCJpYXQiOjE3MDg1NzcyNDAsImV4cCI6MTcwODYwNzI0MCwibmJmIjoxNzA4NTc2NjQwLCJhdWQiOiJbXCJ4OjBcIiwgXCJ4OjFcIiwgXCJ4OjJcIiwgXCJkOjFcIiwgXCJkOjJcIiwgXCJ4OjFcIiwgXCJ4OjBcIl0iLCJzdWIiOiJhdXRoX2NvZGUiLCJkaXNwbGF5X25hbWUiOiJYUzk4MzY4Iiwib21zIjoiSzEiLCJoc21fa2V5IjoiZjRjNjQ5NTdjNTMwMTA5ZDk1ZDhjNmM4MWJhMTcxZWU5NGJmMzI4ZjEwZDdjYzFiZjQyYTUwODEiLCJub25jZSI6IiIsImFwcF9pZCI6IlBQODYwUEJFVTAiLCJ1dWlkIjoiYmVjYjFmMDMxYjU3NDQ2OTkxNDczMzA5OTdiMjg3OTAiLCJpcEFkZHIiOiIwLjAuMC4wIiwic2NvcGUiOiIifQ.HWGWNVZyP2J_fGz3jH0bmKaAA0Ebql51t6CzTmcnpMQ"

In [ ]:
grant_type = "authorization_code"
session = fyersModel.SessionModel(
    client_id=client_id,
    secret_key=secret_key, 
    redirect_uri=redirect_uri, 
    response_type=response_type, 
    grant_type=grant_type
)
session.set_token(auth_code)
response = session.generate_token()
print(response)

In [ ]:
df=pd.DataFrame()
df=response
tk=df['access_token']
print(tk)

In [ ]:
fyers = fyersModel.FyersModel(client_id=client_id, is_async=False, token=tk, log_path="")

In [ ]:
def signal_generator(df):
    Open = df.open.iloc[-1]
    Close = df.close.iloc[-1]
    previous_Open = df.open.iloc[-2]
    previous_Close = df.close.iloc[-2]
    
    # red candle
    if (Open>Close):
        return 1

    # green candle
    elif (Open<Close):
        return 2
    
    # No clear pattern
    else:
        return 0


In [ ]:
call=input(str("Enter the symbol for call :"))

In [ ]:
put=input(str("Enter the symbol for put :"))

In [ ]:
calltight=int(input("Enter qty for tight lots for call option:  "))

In [ ]:
callspare=int(input("Enter qty for spare lots for call option:  "))

In [ ]:
puttight=int(input("Enter qty for tight lots for put option:  "))

In [ ]:
putspare=int(input("Enter qty for spare lots for put option:  "))

In [ ]:
def get_datacall():
    data = {
    "symbol":call,
    "resolution":"5",
    "date_format":"1",
    "range_from":date.today(),
    "range_to":date.today(),
    "cont_flag":"1"
    }

    hdata=fyers.history(data=data)
    df = pd.DataFrame.from_dict(hdata['candles'])
    cols = ['datetime','open','high','low','close','volume']
    df.columns=cols
    df['datetime']= pd.to_datetime(df['datetime'],unit="s")
    df['datetime'] = df['datetime'].dt.tz_localize('utc').dt.tz_convert('Asia/Kolkata')
    df['datetime'] = df['datetime'].dt.tz_localize(None)
    df['date'] = df['datetime'].dt.date
    df['time'] = df['datetime'].dt.time
    
    signal = []
    signal.append(0)
    for i in range(1,len(df)):
        dataF = df[i-1:i+1]
        signal.append(signal_generator(dataF))
    #signal_generator(data)
    df["signal"] = signal
    
    goodpair = pd.DataFrame(columns=['Date', 'Time', 'HC', 'LC', 'ET', 'ES', 'orderID', 'Target', 'SL'])
  
    for i in range(1,len(df)):
        h=0
        l=0
        t=0
        presenthigh = df.high.iloc[i]
        previoushigh = df.high.iloc[i-1]
        presentlow = df.low.iloc[i]
        previouslow = df.low.iloc[i-1]
        previoustime = df.time.iloc[i-1]
        presenttime = df.time.iloc[i]
        previousdate = df.date.iloc[i-1]
        presentdate = df.date.iloc[i]
        signalfirst = df.signal.iloc[i]
        signalsecond = df.signal.iloc[i-1]
        if(signalfirst!=signalsecond): 
    
            if(previoushigh>presenthigh and presentlow<previouslow):
                h = previoushigh
                l = presentlow
                t=1
            elif(previoushigh<presenthigh and presentlow>previouslow):
                h = presenthigh
                l = previouslow
                t=1
            elif(presentlow<previouslow and presenthigh>previoushigh and 0<=(presenthigh-previoushigh)<=0.05):
                h=presenthigh
                l=presentlow
                t=1
            elif(previouslow<presentlow and previoushigh>presenthigh and 0<=(previoushigh-presenthigh)<=0.05):
                h=previoushigh
                l=previouslow
                t=1
            elif(presenthigh==previoushigh and presentlow<previouslow):
                h=presenthigh
                l=presentlow
                t=1
            elif(presenthigh==previoushigh and presentlow>previouslow):
                h=presenthigh
                l=previouslow
                t=1
            elif(presenthigh>previoushigh and presentlow==previouslow):
                h=presenthigh
                l=presentlow
                t=1
            elif(presenthigh<previoushigh and presentlow==previouslow):
                h=previoushigh
                l=presentlow
                t=1
            elif(presenthigh==previoushigh and presentlow==previouslow):
                h=presenthigh
                l=presentlow
                t=1
                
        if(abs(h-l)>=0 and abs(h-l)<=2 and t==1):
            goodpair.loc[len(goodpair)] = [previousdate, previoustime, h, l, 0, 0, 0, 0, 0]
    global goodpairCE
    goodpairCE=goodpair

In [ ]:
get_datacall()
goodpairCE
#150CE

In [ ]:
def get_dataput():
    data = {
    "symbol":put,
    "resolution":"15",
    "date_format":"1",
    "range_from":"2024-01-30",
    "range_to":"2024-01-30",
    "cont_flag":"1"
    }

    hdata=fyers.history(data=data)
    df = pd.DataFrame.from_dict(hdata['candles'])
    cols = ['datetime','open','high','low','close','volume']
    df.columns=cols
    df['datetime']= pd.to_datetime(df['datetime'],unit="s")
    df['datetime'] = df['datetime'].dt.tz_localize('utc').dt.tz_convert('Asia/Kolkata')
    df['datetime'] = df['datetime'].dt.tz_localize(None)
    df['date'] = df['datetime'].dt.date
    df['time'] = df['datetime'].dt.time
    
    signal = []
    signal.append(0)
    for i in range(1,len(df)):
        dataF = df[i-1:i+1]
        signal.append(signal_generator(dataF))
    #signal_generator(data)
    df["signal"] = signal
    
    goodpair = pd.DataFrame(columns=['Date', 'Time', 'HP', 'LP', 'ET', 'ES', 'orderID', 'Target', 'SL'])
    
    for i in range(1,len(df)):
        h=0
        l=0
        t=0
        signalfirst = df.signal.iloc[i]
        signalsecond = df.signal.iloc[i-1]
        presenthigh = df.high.iloc[i]
        previoushigh = df.high.iloc[i-1]
        presentlow = df.low.iloc[i]
        previouslow = df.low.iloc[i-1]
        previoustime = df.time.iloc[i-1]
        presenttime = df.time.iloc[i]
        previousdate = df.date.iloc[i-1]
        presentdate = df.date.iloc[i]
    
        if(signalfirst!=signalsecond):
            if(previoushigh>presenthigh and presentlow<previouslow):
                h = previoushigh
                l = presentlow
                t=1
            elif(previoushigh<presenthigh and presentlow>previouslow):
                h = presenthigh
                l = previouslow
                t=1
            elif(presentlow<previouslow and presenthigh>previoushigh and 0<=(presenthigh-previoushigh)<=0.05):
                h=presenthigh
                l=presentlow
                t=1
            elif(previouslow<presentlow and previoushigh>presenthigh and 0<=(previoushigh-presenthigh)<=0.05):
                h=previoushigh
                l=previouslow
                t=1
            elif(presenthigh==previoushigh and presentlow<previouslow):
                h=presenthigh
                l=presentlow
                t=1
            elif(presenthigh==previoushigh and presentlow>previouslow):
                h=presenthigh
                l=previouslow
                t=1
            elif(presenthigh>previoushigh and presentlow==previouslow):
                h=presenthigh
                l=presentlow
                t=1
            elif(presenthigh<previoushigh and presentlow==previouslow):
                h=previoushigh
                l=presentlow
                t=1
            elif(presenthigh==previoushigh and presentlow==previouslow):
                h=presenthigh
                l=presentlow
                t=1
        if(abs(h-l)>=0 and abs(h-l)<=2 and t==1):
            goodpair.loc[len(goodpair)] = [previousdate, previoustime, h, l, 0, 0, 0, 0, 0]
    global goodpairPE
    goodpairPE=goodpair

In [ ]:
get_dataput()
goodpairPE
#175PE

In [ ]:

#def onmessage(message):
    t = time.localtime()
    cmin = time.strftime("%M", t)
    csec = time.strftime("%S", t)
    if (int(cmin) % 5 == 0 and int(csec)<3):
        get_datacall()
        get_dataput()
    print("Response : ", message)
    if (len(goodpairCE)>=1 and goodpairCE.Date.iloc[-1]==date.today() and goodpairCE.ET.iloc[-1]==0 and message['symbol']==call and message['ltp']>=goodpairCE.HC.iloc[-1]):
        data_ct = {
            "symbol":call,
            "qty":calltight,
            "type":1,
            "side":1,
            "productType":"BO",
            "limitPrice":goodpairCE.HC.iloc[-1],
            "stopPrice":0,
            "validity":"DAY",
            "disclosedQty":0,
            "offlineOrder":False,
            "stopLoss":goodpairCE.LC.iloc[-1],
            "takeProfit":1
        }
        response_ct = fyers.place_order(data=data_ct)
        #print(response_ct)
        if response_ct['s']=='ok':
            print(f"entry tight slot for call {call}")
            goodpairCE.ET.iloc[-1]=1
        
    if (len(goodpairPE)>=1 and goodpairPE.Date.iloc[-1]==date.today() and goodpairPE.ET.iloc[-1]==0 and message['symbol']==put and message['ltp']>=goodpairPE.HP.iloc[-1]):
        data_pt = {
            "symbol":put,
            "qty":puttight,
            "type":1,
            "side":1,
            "productType":"BO",
            "limitPrice":goodpairPE.HP.iloc[-1],
            "stopPrice":0,
            "validity":"DAY",
            "disclosedQty":0,
            "offlineOrder":False,
            "stopLoss":goodpairPE.LP.iloc[-1],
            "takeProfit":1
        }
        response_pt = fyers.place_order(data=data_pt)
        #print(response_pt)
        if response_pt['s']=='ok':
            print(f"entry tight slot for put {put}")
            goodpairPE.ET.iloc[-1]=1
            
    if (len(goodpairCE)>=1 and goodpairCE.Date.iloc[-1]==date.today() and goodpairCE.ES.iloc[-1]==0 and message['symbol']==call and message['ltp']>=goodpairCE.HC.iloc[-1]):
        data_cs = {
            "symbol":call,
            "qty":callspare,
            "type":1,
            "side":1,
            "productType":"CO",
            "limitPrice":goodpairCE.HC.iloc[-1],
            "stopPrice":0,
            "validity":"DAY",
            "disclosedQty":0,
            "offlineOrder":False,
            "stopLoss":goodpairCE.LC.iloc[-1],
            "takeProfit":0
        }
        response_cs = fyers.place_order(data=data_cs)
        #print(response_cs)
        if response_cs['s']=='ok':
            print(f"entry spare slot for call {call}")
            goodpairCE.orderID.iloc[-1]=response_cs['id']
            goodpairCE.Target.iloc[-1]=goodpairCE.HC.iloc[-1]+15
            goodpairCE.SL.iloc[-1]=goodpairCE.LC.iloc[-1]
            goodpairCE.ES.iloc[-1]=1
            
    for i in range(len(goodpairCE)):
        if(goodpairCE.ES.iloc[i]==1 and message['symbol']==call and message['ltp']>=goodpairCE.Target.iloc[i]):
            goodpairCE.Target.iloc[i]+=15
            goodpairCE.SL.iloc[i]=goodpairCE.Target.iloc[i]-30
            orderIdC=goodpairCE.orderID.iloc[i]
            data_mcs = {
                "id":orderIdC, 
                "type":1, 
                "stopPrice":goodpairCE.SL.iloc[i],
                "qty":callspare
            }
            response_mcs = fyers.modify_order(data=data_mcs)
            if response_mcs['s']=='ok':
                print(f"spare slot trailed for call={call}, orderID={goodpairCE.orderID.iloc[i]} at sl={goodpairCE.SL.iloc[i]}")
            
    if (len(goodpairPE)>=1 and goodpairPE.Date.iloc[-1]==date.today() and goodpairPE.ES.iloc[-1]==0 and message['symbol']==put and message['ltp']>=goodpairPE.HP.iloc[-1]):
        data_ps = {
            "symbol":put,
            "qty":putspare,
            "type":1,
            "side":1,
            "productType":"CO",
            "limitPrice":goodpairPE.HP.iloc[-1],
            "stopPrice":0,
            "validity":"DAY",
            "disclosedQty":0,
            "offlineOrder":False,
            "stopLoss":goodpairPE.LP.iloc[-1],
            "takeProfit":0
        }
        response_ps = fyers.place_order(data=data_ps)
        #print(response_ps)
        if response_ps['s']=='ok':
            print(f"entry spare slot for put {put}")
            goodpairPE.orderID.iloc[-1]=response_ps['id']
            goodpairPE.Target.iloc[-1]=goodpairPE.HP.iloc[-1]+15
            goodpairPE.SL.iloc[-1]=goodpairPE.LP.iloc[-1]
            goodpairPE.ES.iloc[-1]=1
            
    for i in range(len(goodpairPE)):
        if(goodpairPE.ES.iloc[-1]==1 and message['symbol']==put and message['ltp']>=goodpairPE.Target.iloc[i]):
            goodpairPE.Target.iloc[i]+=15
            goodpairPE.SL.iloc[i]=goodpairPE.Target.iloc[i]-30
            orderIdP=goodpairPE.orderID.iloc[i]
            data_mps = {
                "id":orderIdP, 
                "type":1, 
                "stopPrice": goodpairPE.SL.iloc[i],
                "qty":putspare
            }
            response_mps = fyers.modify_order(data=data_mps)
            if response_mps['s']=='ok':
                print(f"spare slot trailed for put={put}, orderID={goodpairPE.orderID.iloc[i]} at sl={goodpairPE.SL.iloc[i]}")

def onerror(message):
    print("Error:", message)

def onclose(message):
    print("Connection closed:", message)

def onopen():
    data_type = "SymbolUpdate"
    symbols = [call,put]
    #print(len(goodpairPE))
    ofyers.subscribe(symbols=symbols, data_type=data_type)
    ofyers.keep_running()
access_token = client_id + ":" + tk

ofyers = data_ws.FyersDataSocket(
    access_token=access_token,
    log_path="",                     
    litemode=False,
    write_to_file=False,              
    reconnect=True,                 
    on_connect=onopen,               
    on_close=onclose,                
    on_error=onerror,               
    on_message=onmessage             
)
ofyers.connect()